# Fourier transform of mount errors

Craig Lage - 27-Nov-22

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from astro_metadata_translator import ObservationInfo
from lsst_efd_client import merge_packed_time_series as mpts

from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
logger = DecoratedLogger.get_decorated_logger()

In [ ]:
NON_TRACKING_IMAGE_TYPES = ['BIAS',
                            'FLAT',
                            ]

AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC = 280.0
MOUNT_IMAGE_WARNING_LEVEL = .25  # this determines the colouring of the cells in the table, yellow for this
MOUNT_IMAGE_BAD_LEVEL = .4


def _getEfdData(client, dataSeries, startTime, endTime):
    """A synchronous warpper for geting the data from the EFD.

    This exists so that the top level functions don't all have to be async def.
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(client.select_time_series(dataSeries, ['*'], startTime.utc, endTime.utc))

In [ ]:
def calculateFFTPeaks(dataId, butler, client):
    """Queries EFD for a given exposure and calculates the RMS errors in the
    axes during the exposure, optionally plotting and saving the data.

    Returns ``False`` if the analysis fails or is skipped e.g. due to the
    expTime being too short.

    Parameters
    ----------
    dataId : `dict` or `lsst.daf.butler.DataCoordinate`
        The dataId for which to plot the mount torques.
    butler : `lsst.daf.butler.Butler`
        The butler to use to retrieve the image metadata.
    client : `lsst_efd_client.Client`
        The EFD client to retrieve the mount torques.
    figure : `matplotlib.figure.Figure`
        A matplotlib figure to re-use. Necessary to pass this in to prevent an
        ever-growing figure count and the ensuing memory leak.
    saveFilename : `str`
        Full path and filename to save the plot to.
    logger : `logging.Logger`
        The logger.

    Returns
    -------
    axisErrors : `dict` or `False`
        The RMS errors in the three axes and their image contributions:
        ``az_rms`` - The RMS azimuth error.
        ``el_rms`` - The RMS elevation error.
        ``rot_rms`` - The RMS rotator error.
        ``image_az_rms`` - The RMS azimuth error for the image.
        ``image_el_rms`` - The RMS elevation error for the image.
        ``image_rot_rms`` - The RMS rotator error for the image.
    """
    # lsst-efd-client is not a required import at the top here, but is
    # implicitly required as a client is passed into this function so is not
    # rechecked here.

    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    dayString = dayObsIntToString(expRecord.day_obs)
    seqNumString = str(expRecord.seq_num)
    dataIdString = f"{dayString} - seqNum {seqNumString}"

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        return False

    exptime = expRecord.exposure_time
    if exptime < 1.99:
        return False

    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    elevation = 90 - expRecord.zenith_angle

    # TODO: DM-33859 remove this once it can be got from the expRecord
    md = butler.get('raw.metadata', dataId, detector=0)
    obsInfo = ObservationInfo(md)
    azimuth = obsInfo.altaz_begin.az.value
    # Time base in the EFD is still a big mess.  Although these times are in
    # UTC, it is necessary to tell the code they are in TAI. Then it is
    # necessary to tell the merge_packed_time_series to use UTC.
    # After doing all of this, there is still a 2 second offset,
    # which is discussed in JIRA ticket DM-29243, but not understood.

    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')

    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    nasmyth_position = _getEfdData(client, "lsst.sal.ATMCS.mount_Nasmyth_Encoders", t_start, t_end)
    torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)

    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)
    rot = mpts(nasmyth_position, 'nasmyth2CalculatedAngle', stride=1)
    az_torque_1 = mpts(torques, 'azimuthMotor1Torque', stride=1)
    az_torque_2 = mpts(torques, 'azimuthMotor2Torque', stride=1)
    el_torque = mpts(torques, 'elevationMotorTorque', stride=1)
    rot_torque = mpts(torques, 'nasmyth2MotorTorque', stride=1)

    # Calculate the tracking errors
    az_vals = np.array(az.values[:, 0])
    el_vals = np.array(el.values[:, 0])
    rot_vals = np.array(rot.values[:, 0])
    times = np.array(az.values[:, 1])
    # The fits are much better if the time variable
    # is centered in the interval
    fit_times = times - times[int(len(az.values[:, 1]) / 2)]

    # Fit with a polynomial
    az_fit = np.polyfit(fit_times, az_vals, 4)
    el_fit = np.polyfit(fit_times, el_vals, 4)
    rot_fit = np.polyfit(fit_times, rot_vals, 2)
    az_model = np.polyval(az_fit, fit_times)
    el_model = np.polyval(el_fit, fit_times)
    rot_model = np.polyval(rot_fit, fit_times)

    # Errors in arcseconds
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600

    # Calculate RMS
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    # Calculate Image impact RMS
    image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
    image_el_rms = el_rms
    image_rot_rms = rot_rms * AUXTEL_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0

    # Calculate the FFT peaks
    fft_peaks = []
    for i, error in enumerate([az_error, el_error]):
        # Number of samples in normalized_tone
        N = len(error)
        SAMPLE_RATE = 100 # Samples/sec
        
        yf = fft(error)
        yf = yf[0:int(len(az_error)/2)]
        xf = fftfreq(N, 1 / SAMPLE_RATE)
        xf = xf[0:int(len(error)/2)]
        yf = np.abs(fft(error))
        yf = yf[0:int(len(error)/2)]
        max = np.max(yf)
        peak_indices, peak_dict = find_peaks(yf, height=max/100) 
        peak_heights = peak_dict['peak_heights']
        
        for j in range(1,4):
            peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
            peak_freq = xf[peak_index]
            height_index = np.where(peak_indices == peak_index)[0][0]
            peak_height = peak_heights[height_index]
            fft_peaks.append([peak_freq, peak_height])
    return fft_peaks

    


In [ ]:
expId = 2023110800415 # Oscillation
#expId = 2023111600552 # Wind
#expId = 2023111600561 # Crazy mount?
#expId = 2023112000238 # Crazy mount?
#expId = 2023112000201 # Shutter open too soon
#expId = 2023110700594 # Timebase errors 1
#expId = 2023110700519 # Timebase errors 2
dataId = {'detector':0, 'exposure':expId}
fft_peaks = calculateFFTPeaks(dataId, butler, client)
print(fft_peaks)

In [ ]:
az_error = err['az_error']
el_error = err['el_error']
times = err['fit_times']

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(8,4))
for i, error in enumerate([az_error, el_error]):
    # Number of samples in normalized_tone
    N = len(error)
    SAMPLE_RATE = 100 # Samples/sec
    
    yf = fft(error)
    yf = yf[0:int(len(az_error)/2)]
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    xf = xf[0:int(len(error)/2)]
    yf = np.abs(fft(error))
    yf = yf[0:int(len(error)/2)]
    max = np.max(yf)
    peak_indices, peak_dict = find_peaks(yf, height=max/100) 
    peak_heights = peak_dict['peak_heights']
    for j in range(1,4):
        try:
            peak_index = peak_indices[np.argpartition(peak_heights,-j)[-j]]
            xplot = xf[peak_index]
            height_index = np.where(peak_indices == peak_index)[0][0]
            yplot = peak_heights[height_index]
            print(peak_index, xplot, yplot)
            axs[i].plot([xplot,xplot], [0,yplot], ls='--', color='black')
        except:
            continue
        
    axs[i].plot(xf, yf)
    axs[i].set_xlim(0,10)
    #axs[i].set_ylim(0,500)
